# Webscraping qury

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv('/workspaces/ml-learning/.env', override=True)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test initialization

In [4]:
from phages.modules.library import Library
from llama_index.llms import OpenAI
library = Library().withLLM(OpenAI(model='gpt-4'))

/workspaces/ml-learning/src/phages/modules/library.py:69: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator('source', pre=True, always=True)


embedded weaviate is already listening on port 8079


{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"docs_7hK4rmDaZn7X","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-28T03:54:23Z","took":68200}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"nodes_F9gFGkEajvtV","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-28T03:54:23Z","took":68100}


In [5]:
library.client.is_ready()

True

In [28]:
library.client.schema.get()

{'classes': [{'class': 'Docs',
   'description': 'Class for Docs',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'Text property',
     'indexFilterable': True,
     'indexSearchable': True,
     'name': 'text',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'The ref_doc_id of the Node',
     'indexFilterable': True,
     'indexSearchable': True,
     'name': 'ref_doc_id',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'node_info (in JSON)',
     'indexFilterable': True,
     'indexSearchable': True,
     'name': 'node_info',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'The relationships of the node (in JSON)',
     'indexFilterable': True,
     'indexSearchable': True,
   

In [27]:
library.client.data_object.get(class_name='Docs', with_vector=False)

{'deprecations': [],
 'objects': [{'class': 'Docs',
   'creationTimeUnix': 1706414989023,
   'id': '390ec987-3df0-4711-ad7e-dc6e7295dff6',
   'lastUpdateTimeUnix': 1706414989023,
   'properties': {'_node_content': '{"id_": "390ec987-3df0-4711-ad7e-dc6e7295dff6", "embedding": null, "metadata": {"file_name": "/workspaces/ml-learning/src/phages/data/test3/fmicb-09-00850.pdf", "citation": "Zhou, W., Feng, Y., & Zong, Z. (2018). Two New Lytic Bacteriophages of the Myoviridae Family Against Carbapenem-Resistant Acinetobacter baumannii. Frontiers in Microbiology, 9, 850. doi:10.3389/fmicb.2018.00850"}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "text": "", "start_char_idx": null, "end_char_idx": null, "text_template": "{metadata_str}\\n\\n{content}", "metadata_template": "{key}: {value}", "metadata_seperator": "\\n", "class_name": "TextNode"}',
    '_node_type': 'TextNode',
    'citation': 'Zhou, W., Feng, Y., & Zong, Z. (2018). Two New Lytic Ba

In [ ]:
library.docs_index.as_query_engine()

In [15]:
try:    
    library.add("some/random/url")
except (ValueError, Exception) as e:
    print(f"Blob {e}")
    

Blob 1 validation error for SourceType
source
  Value error, The source must be a valid URL or an existing local file path. [type=value_error, input_value='some/random/url', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


## Test querying 5 files

In [9]:
# add 3 microbiology pdf
from pathlib import Path

def find_files(directory, ext = '*.pdf'):
    # Convert the directory string to a Path object
    path = Path(directory)

    # Check if the provided path is a directory
    if not path.is_dir():
        raise ValueError("Provided path is not a directory")

    # Loop through the directory
    for file in path.glob(ext):
        # Each 'file' is a Path object representing a PDF file
        yield str(file)


for file in find_files("/workspaces/ml-learning/src/phages/data/test3/"):
    library.add(file)

Multiple definitions in dictionary at byte 0x32487 for key /MediaBox
Multiple definitions in dictionary at byte 0x32487 for key /MediaBox
Multiple definitions in dictionary at byte 0x327a8 for key /MediaBox
Multiple definitions in dictionary at byte 0x327a8 for key /MediaBox
Multiple definitions in dictionary at byte 0x32a79 for key /MediaBox
Multiple definitions in dictionary at byte 0x32a79 for key /MediaBox
Multiple definitions in dictionary at byte 0x32c82 for key /MediaBox
Multiple definitions in dictionary at byte 0x32c82 for key /MediaBox
Multiple definitions in dictionary at byte 0x32e3c for key /MediaBox
Multiple definitions in dictionary at byte 0x32e3c for key /MediaBox
Multiple definitions in dictionary at byte 0x32fee for key /MediaBox
Multiple definitions in dictionary at byte 0x32fee for key /MediaBox
Multiple definitions in dictionary at byte 0x331d8 for key /MediaBox
Multiple definitions in dictionary at byte 0x331d8 for key /MediaBox
Multiple definitions in dictionary

In [10]:
query = 'What are the bacteriophages(s) that show promising results in aquaculture and food industries and what is the pathogen that each can control?'

In [11]:
response_adoc_match = await library._adoc_match(query)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [12]:
from pprint import pprint
pprint(response_adoc_match)

[NodeWithScore(node=TextNode(id_='5eb07d54-a7f3-4e5b-b8dd-0677bc1ee2ee', embedding=[-0.0067372723, -0.018468704, -0.005094457, -0.024804784, -0.0040084696, 0.012457728, -0.024514263, 0.0041675633, -0.010507101, -0.013481462, 0.016905434, 0.03962125, -0.0132186115, -0.0048108553, -0.010763035, 0.0020837816, 0.029383915, -0.029549927, 0.021941097, -0.03638404, 0.006685394, 0.009946815, -0.0061493176, 0.00017249562, 0.0026233168, 0.012990346, 0.008279789, -0.019049741, -0.014844134, 0.0014223323, 0.0019454393, -0.0030590952, -0.007954685, 0.011752183, -0.016933104, -0.026824582, 0.02855386, -0.006132025, 0.029162567, 0.009580208, 0.006996664, -0.008964584, -0.0002842503, 0.0041191434, 0.005606324, 0.00957329, 0.010832206, -0.007062377, -0.020460833, 0.012637573, 0.05334481, -0.014525947, 0.0014474068, 0.03148672, -0.009607876, -0.0034135976, 0.016241392, 0.004548005, -0.0063395384, -0.01936793, -0.0018243897, -0.008487303, -0.019312592, -0.025966858, 0.0012113602, -0.012312469, -0.0027080

In [47]:
len(response_adoc_match)

5

In [13]:
for r in response_adoc_match:
    print(r.text)
    print(r.metadata)
    print(r.score)

Tan, C. W., Rukayadi, Y., Hasan, H., Abdul-Mutalib, N.-A., Jambari, N. N., Hara, H., Thung, T. Y., Lee, E., & Radu, S. (2021). Isolation and Characterization of Six Vibrio parahaemolyticus Lytic Bacteriophages From Seafood Samples. Frontiers in Microbiology, 12, 616548. doi: 10.3389/fmicb.2021.616548
{'file_name': '/workspaces/ml-learning/src/phages/data/test3/fmicb-12-616548.pdf', 'citation': 'Tan, C. W., Rukayadi, Y., Hasan, H., Abdul-Mutalib, N.-A., Jambari, N. N., Hara, H., Thung, T. Y., Lee, E., & Radu, S. (2021). Isolation and Characterization of Six Vibrio parahaemolyticus Lytic Bacteriophages From Seafood Samples. Frontiers in Microbiology, 12, 616548. doi: 10.3389/fmicb.2021.616548'}
0.83827609
Liao, Yen-Te, et al. "Discovery of Shiga Toxin-Producing Escherichia coli (STEC)-Specific Bacteriophages From Non-fecal Composts Using Genomic Characterization." Frontiers in Microbiology, vol. 10, 2019, p. 627. doi: 10.3389/fmicb.2019.00627.
{'file_name': '/workspaces/ml-learning/src/p

In [53]:
# library.docs_index.docstore.persist('./storage/phages_docs_index_docstore.json')

In [49]:
# library.docs_index.vector_store.persist('./storage/phages_docs_index_vector_store.json')

## Testing aquery

In [14]:
response_aquery = await library.aquery(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [19]:
from pprint import pprint
pprint(response_aquery.query)
pprint(response_aquery.answer)
pprint(set([c.node.metadata['citation'] for c in response_aquery.contexts]))

('What are the bacteriophages(s) that show promising results in aquaculture '
 'and food industries and what is the pathogen that each can control?')
('Bacteriophages Vp33, Vp22, Vp21, Vp02, Vp08, and Vp11 have shown promising '
 'results in controlling the pathogen Vibrio parahaemolyticus, which is a '
 'leading foodborne pathogen often isolated from a variety of seafood. These '
 'bacteriophages were isolated from seafood samples and characterized based on '
 'host specificity, morphological, genomic character, protein profile, and '
 'temperature and pH stability. The in vitro lytic activity of these '
 'bacteriophages demonstrated significant effects against V. parahaemolyticus. '
 'Furthermore, the application of a bacteriophage cocktail instead of a single '
 'bacteriophage suspension was observed to have better efficiency in '
 'controlling the growth of V. parahaemolyticus (Tan et al., 2021).')
{'Tan, C. W., Rukayadi, Y., Hasan, H., Abdul-Mutalib, N.-A., Jambari, N. N., '
 'Har

In [20]:
library.add('/workspaces/ml-learning/src/phages/data/test3/aem.01468-21.pdf')

New documents count: 1
calling get_citation
citation prompt: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['year', 'text'] kwargs={'year': 2024} output_parser=None template_var_mappings=None function_mappings=None template='Provide the citation for the following text in MLA Format. The year is {year}\n\n    {text}\n\n\n    Citation:'
text: Isolation of a Virulent Aeromonas salmonicida subsp. masoucida
Bacteriophage and Its Application in Phage Therapy in Turbot
(Scophthalmus maximus )
Zhenhe Xu ,aPeng Jin ,aXiangshan Zhou ,aYuanxing Zhang ,a,bQiyao Wang ,a,cXiaohong Liu ,a,cShuai Shao ,a,c
Qin Liua,c
aState Key Laboratory of Bioreactor Engineering, East China University of Science and Technology, Shanghai, China
bSouthern Marine Science and Engineering Guangdong Laboratory (Zhuhai), Zhuhai, China
cShanghai Engineering Research Center of Maricultured Animal Vaccines, Shanghai, China
ABSTRACT Aeromonas salmonicida is an aquatic pathogen that can infect a variety 

True

In [21]:
response_aquery = await library.aquery(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [22]:
from pprint import pprint
pprint(response_aquery.query)
pprint(response_aquery.answer)
pprint(set([c.node.metadata['citation'] for c in response_aquery.contexts]))

('What are the bacteriophages(s) that show promising results in aquaculture '
 'and food industries and what is the pathogen that each can control?')
('Bacteriophages have shown promising results in controlling bacterial '
 'pathogens in the aquaculture and food industries. A study isolated three A. '
 'salmonicida subsp. masoucida phages from sewage, one of which, vB_AsM_ZHF, '
 'was found to inhibit A. salmonicida subsp. masoucida infection in vitro and '
 'in vivo, suggesting it could be an alternative to antibiotics for protecting '
 'fish against multidrug-resistant A. salmonicida subsp. masoucida (Xu, '
 'Zhenhe, et al., 2024). Commercial phage products such as Salmonlex, Listex, '
 'Listshield, and Ecoshield have been reported to significantly reduce the '
 'number of pathogens including Salmonella, Escherichia coli, and Listeria '
 'monocytogenes during food storage (Xu, Zhenhe, et al., 2024). Additionally, '
 'six Vibrio parahaemolyticus lytic bacteriophages isolated from seaf

In [74]:
pprint([n for n in response_aquery.selected_documents])

[NodeWithScore(node=TextNode(id_='57c9d900-e1b0-4704-8e01-f0a011f9db7f', embedding=[-0.0067372723, -0.018468704, -0.005094457, -0.024804784, -0.0040084696, 0.012457728, -0.024514263, 0.0041675633, -0.010507101, -0.013481462, 0.016905434, 0.03962125, -0.0132186115, -0.0048108553, -0.010763035, 0.0020837816, 0.029383915, -0.029549927, 0.021941097, -0.03638404, 0.006685394, 0.009946815, -0.0061493176, 0.00017249562, 0.0026233168, 0.012990346, 0.008279789, -0.019049741, -0.014844134, 0.0014223323, 0.0019454393, -0.0030590952, -0.007954685, 0.011752183, -0.016933104, -0.026824582, 0.02855386, -0.006132025, 0.029162567, 0.009580208, 0.006996664, -0.008964584, -0.0002842503, 0.0041191434, 0.005606324, 0.00957329, 0.010832206, -0.007062377, -0.020460833, 0.012637573, 0.05334481, -0.014525947, 0.0014474068, 0.03148672, -0.009607876, -0.0034135976, 0.016241392, 0.004548005, -0.0063395384, -0.01936793, -0.0018243897, -0.008487303, -0.019312592, -0.025966858, 0.0012113602, -0.012312469, -0.0027080

In [ ]:
pprint([n.text for n in response_aquery.contexts])

In [77]:
for cit in set([n.metadata['citation'] for n in response_aquery.contexts]):
    pprint(cit)

('Sharma, Ruchira, et al. "A Novel, Highly Related Jumbo Family of '
 'Bacteriophages That Were Isolated Against Erwinia." Frontiers in '
 'Microbiology, vol. 10, 2019, p. 1533. doi: 10.3389/fmicb.2019.01533.')
('Tan, C. W., Rukayadi, Y., Hasan, H., Abdul-Mutalib, N.-A., Jambari, N. N., '
 'Hara, H., Thung, T. Y., Lee, E., & Radu, S. (2021). Isolation and '
 'Characterization of Six Vibrio parahaemolyticus Lytic Bacteriophages From '
 'Seafood Samples. Frontiers in Microbiology, 12, 616548. doi: '
 '10.3389/fmicb.2021.616548')


In [3]:
added = library.add("/workspaces/ml-learning/src/phages/data/test1/stateoftheunion.txt")
added

New documents count: 1
calling get_citation
citation prompt: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['year', 'text'] kwargs={'year': 2024} output_parser=None template_var_mappings=None function_mappings=None template='Provide the citation for the following text in MLA Format. The year is {year}\n\n    {text}\n\n\n    Citation:'
text: Mr. Speaker. Madam Vice President. Our First Lady and Second Gentleman.

Members of Congress and the Cabinet. Leaders of our military.

Mr. Chief Justice, Associate Justices, and retired Justices of the Supreme Court.

And you, my fellow Americans.

I start tonight by congratulating the members of the 118th Congress and the new Speaker of the House, Kevin McCarthy.

Mr. Speaker, I look forward to working together.

I also want to congratulate the new leader of the House Democrats and the first Black House Minority Leader in history, Hakeem Jeffries.

Congratulations to the longest serving Senate Leader in history, Mitch McCon

True

In [37]:
ret = library.docs_index.as_retriever()
ret.retrieve("Speech of Biden")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='512a1f3c-ea8f-482a-bd6a-5a05cc169586', embedding=[-0.020383993, -0.010523272, -0.007901264, 0.009381429, 0.003897769, 0.00027554866, -0.007633425, -0.006512728, -0.028109048, 0.011918856, 0.046012, 0.015548785, -0.023386614, 0.008641347, -0.010678336, -0.0028687017, -0.002271349, -0.019495893, 0.008796412, -0.032817382, 0.008775267, 0.006502155, -0.015478302, 0.03653894, 0.008479234, -0.030872023, 0.018015727, -0.026769849, -0.008288926, -0.017057145, 0.0021074736, -0.019679151, -0.015295043, -0.010713578, -0.007795538, -0.022540806, -0.0038871963, -0.03216893, 0.0389354, -0.015548785, 0.009212268, -0.0043946817, -0.009395527, -0.0035206792, -0.029490534, 0.0030167182, 0.010628997, -0.021229802, -0.0066466476, 0.013159376, 0.014745267, 0.02194874, -0.016521465, -0.0025955758, -0.0004955911, 0.0028017417, -0.0072316653, 0.0047048116, -0.027883498, 0.0032986545, 0.01588711, -0.019834217, -0.003809664, -0.0013251005, -0.00703431, -0.031153958, -0.00858495

In [25]:
library.docs_index.todict()

AttributeError: 'VectorStoreIndex' object has no attribute 'todict'

In [14]:
for i in library.documents:
    print(i.metadata['citation'])

"Transcript: President Biden's Address to Congress." The New York Times, 24 Apr. 2024, www.nytimes.com/2024/04/24/us/politics/biden-speech-transcript.html.


In [12]:
len(library.nodes)

0

In [15]:
import pprint
pprint.pprint(library.documents[0].metadata)

{'citation': '"Transcript: President Biden\'s Address to Congress." The New '
             'York Times, 24 Apr. 2024, '
             'www.nytimes.com/2024/04/24/us/politics/biden-speech-transcript.html.',
 'creation_date': '2024-01-07',
 'file_name': 'stateoftheunion.txt',
 'file_path': '/workspaces/ml-learning/src/phages/data/test1/stateoftheunion.txt',
 'file_size': 43412,
 'file_type': 'text/plain',
 'last_accessed_date': '2024-01-07',
 'last_modified_date': '2024-01-07'}


In [17]:
pprint.pprint(library.nodes[0].metadata)

{'citation': '"Transcript: President Biden\'s Address to Congress." The New '
             'York Times, 24 Apr. 2024, '
             'www.nytimes.com/2024/04/24/us/politics/biden-speech-transcript.html.',
 'creation_date': '2024-01-07',
 'excerpt_keywords': 'Speaker, Vice President, First Lady, Second Gentleman, '
                     'Congress, Cabinet, military, Chief Justice, Associate '
                     'Justices, retired Justices, Supreme Court, Americans, '
                     '118th Congress, Kevin McCarthy, House Democrats, Hakeem '
                     'Jeffries, Senate Leader, Mitch McConnell, Chuck Schumer, '
                     'Nancy Pelosi, progress, resilience, economy, jobs, '
                     'COVID, democracy, crisis, possibilities, Democrats, '
                     'Republicans, Europe, infrastructure law, veterans, '
                     'bipartisan laws, soul of the nation, middle class, '
                     'manufacturing jobs, pride, dignity, unemploy

In [4]:
library.add("https://arxiv.org/ftp/arxiv/papers/1406/1406.4214.pdf")

New documents count: 1
calling get_citation
citation prompt: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['year', 'text'] kwargs={'year': 2024} output_parser=None template_var_mappings=None function_mappings=None template='Provide the citation for the following text in MLA Format. The year is {year}\n\n    {text}\n\n\n    Citation:'
text:  1 Title: Nanopore sequencing of the phi X 174 genome  
 
Authors: Andrew H. Laszlo1, Ian M. Derrington1, Brian C. Ross1, Henry Brinkerhoff1, Andrew  
Adey2, Ian C. Nova1, Jonathan M. Craig1, Kyle W. Langford1, Jenny Mae Samson1, Riza Daza2, 
Kenji Doering1, Jay Shendure2, Jens H. Gundlach1,† 
 
Affiliations: 1Department of Physics, University of Washington, Seattle, WA 98195, USA  
2Department of Genome Sci ences, University of Washington, Seattle, WA 98195, USA  
†Corresponding author. E -mail: gundlach@uw.edu  
 
Abstract : Nanopore sequencing of DNA is a single -molecule technique that may achieve long 
reads, low cost, a

True

In [46]:
library.add("/workspaces/ml-learning/src/phages/data/test1/fmicb-13-938616.pdf")

New documents count: 1
calling get_citation
citation prompt: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['year', 'text'] kwargs={'year': 2024} output_parser=None template_var_mappings=None function_mappings=None template='Provide the citation for the following text in MLA Format. The year is {year}\n\n    {text}\n\n\n    Citation:'
text: fmicb-13-938616 August 10, 2022 Time: 14:20 # 1
TYPE Original Research
PUBLISHED 16 August 2022
DOI10.3389/fmicb.2022.938616
OPEN ACCESS
EDITED BY
Hany Anany,
Agriculture and Agri-Food Canada
(AAFC), Canada
REVIEWED BY
Andrea Isabel Moreno Switt,
Pontiﬁcia Universidad Católica
de Chile, Chile
Carlos E. Martinez-Soto,
University of Guelph, Canada
*CORRESPONDENCE
Qiang Pan
panqiang@phagepharm.com
Huiying Ren
renren0228@sina.com
†These authors have contributed
equally to this work
SPECIALTY SECTION
This article was submitted to
Phage Biology,
a section of the journal
Frontiers in Microbiology
RECEIVED 07 May 2022
ACCEPTED 18 Jul

True

In [5]:
nodes_engine = library.nodes_index.as_query_engine()
response = nodes_engine.query("What do you know about nanopore sequencing?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [6]:
display(Markdown(response.response))

Nanopore sequencing is a single-molecule technique for DNA sequencing that has the potential to achieve long reads, low cost, and high speed with minimal sample preparation and instrumentation. It involves the use of a nanopore device, which consists of a thin membrane with a nanometer-scale pore that separates two salt solution wells. When a voltage is applied across the membrane, ion current flows through the pore, and when single-stranded DNA enters the pore, it blocks some fraction of the ion current. The fraction of the ion current that is blocked depends on the identity of nucleotides within the pore. Recent advancements in nanopore resolution and DNA control have enabled single-nucleotide resolution and control of DNA translocation. However, despite nearly 20 years of effort, nanopore-derived sequence reads that align to complex, natural DNA sequences have yet to be demonstrated.

In [8]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='fcb50563-f2bb-4019-90e6-32c3f86f6c77', embedding=[-0.023770344, -0.016172517, -0.018628243, 0.005633924, -0.007645313, 0.021178942, -0.006203761, -0.0056576673, -0.040349886, -0.015901165, 0.0027966106, 0.015507706, -0.002874624, 0.012041198, 0.011898739, 0.0013414912, 0.011735928, -0.036225352, -0.002491341, -0.009639743, -0.032996275, -0.012366819, 0.010643741, -0.0025066044, -0.004677412, 0.0072654216, 0.023946721, -0.019659376, -0.034760058, -0.004785952, -0.0012185354, 0.011315335, -0.019808618, 0.0007881897, 0.019455863, -0.009972148, 0.015019275, -0.0022861317, 0.03391887, -0.03845043, 0.029007416, 0.0013915215, 0.010189228, 0.0032833465, -0.021178942, -0.018221216, 0.012346468, 0.008147312, -0.012509279, -0.007462151, 0.011993712, 0.01355398, 0.010589471, -0.00054694177, -0.023322614, -0.029143091, 0.014503708, -0.027514987, -0.0117020095, -0.013119819, 0.005386316, 0.0033342247, -0.029197361, 0.020921158, -0.028247634, 0.014557978, 0.015629815

In [9]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(vars(response))


{   'metadata': {   '9fe90d42-9f20-4967-b144-96163b8329ee': {   'citation': 'Laszlo, '
                                                                            'Andrew '
                                                                            'H., '
                                                                            'et '
                                                                            'al. '
                                                                            '"Nanopore '
                                                                            'sequencing '
                                                                            'of '
                                                                            'the '
                                                                            'phi '
                                                                            'X '
                                                                       

In [48]:
display(Markdown(library.documents[2].text[:4000]))

fmicb-13-938616 August 10, 2022 Time: 14:20 # 1
TYPE Original Research
PUBLISHED 16 August 2022
DOI10.3389/fmicb.2022.938616
OPEN ACCESS
EDITED BY
Hany Anany,
Agriculture and Agri-Food Canada
(AAFC), Canada
REVIEWED BY
Andrea Isabel Moreno Switt,
Pontiﬁcia Universidad Católica
de Chile, Chile
Carlos E. Martinez-Soto,
University of Guelph, Canada
*CORRESPONDENCE
Qiang Pan
panqiang@phagepharm.com
Huiying Ren
renren0228@sina.com
†These authors have contributed
equally to this work
SPECIALTY SECTION
This article was submitted to
Phage Biology,
a section of the journal
Frontiers in Microbiology
RECEIVED 07 May 2022
ACCEPTED 18 July 2022
PUBLISHED 16 August 2022
CITATION
Liu W, Han L, Song P, Sun H, Zhang C,
Zou L, Cui J, Pan Q and Ren H (2022)
Complete genome sequencing of a
Tequintavirus bacteriophage with
a broad host range against Salmonella
Abortus equi isolates from donkeys.
Front. Microbiol. 13:938616.
doi: 10.3389/fmicb.2022.938616
COPYRIGHT
© 2022 Liu, Han, Song, Sun, Zhang,
Zou, Cui, Pan and Ren. This is an
open-access article distributed under
the terms of the Creative Commons
Attribution License (CC BY). The use,
distribution or reproduction in other
forums is permitted, provided the
original author(s) and the copyright
owner(s) are credited and that the
original publication in this journal is
cited, in accordance with accepted
academic practice. No use, distribution
or reproduction is permitted which
does not comply with these terms.Complete genome sequencing
of aTequintavirus bacteriophage
with a broad host range against
Salmonella Abortus equi isolates
from donkeys
Wenhua Liu1†, Letian Han1†, Peng Song1†, Huzhi Sun2,
Can Zhang1, Ling Zou1, Jiaqi Cui1, Qiang Pan2*and
Huiying Ren1*
1College of Veterinary Medicine, Qingdao Agricultural University, Qingdao, Shandong, China,
2Qingdao Phagepharm Bio-tech Co., Ltd., Qingdao, Shandong, China
Salmonella enterica subspecies enterica serovar abortus equi (S. Abortus equi)
is the most common cause of abortion in mares. It has recently been found
to cause abortion in donkeys more frequently in China. A novel virulent
bacteriophage vB_SabS_Sds2 (hereafter designated as Sds2) was isolated from
the feces of donkeys using a S. Abortus equi strain as a host. Phage Sds2
had an isometric polyhedral head and an uncontracted long tail, belonging
to the Tequintavirus, Markadamsvirinae, Demerecviridae, Caudovirales. The
genome of phage Sds2 was 114,770 bp, with a GC content of 40.26%. The
genome contained 160 open reading frames (ORFs), and no ORFs were
associated with pathogenicity, drug resistance, or lysogenization by sequence
analysis. Both genome annotation and phylogenetic analysis indicated that
phage Sds2 was highly similar to T5-like bacteriophages. Phage Sds2 could
lyse 100% (30/30) of S. Abortus equi strains, 25.3% (24/95) of other serotypes
ofSalmonella strains, and 27.6% (8/29) of Escherichia coli strains using the
double-layer agar plate method. The in vitro test showed that phage Sds2 had
high bactericidal activity against S. Abortus equi at a wide range of MOIs. The
in vivo test indicated that phage Sds2 had an inhibitory effect on abortion in
mice challenged with S. Abortus equi. In general, phage Sds2 is a novel lytic
phage with a wide host range and has the potential to prevent abortion caused
byS. Abortus equi.
KEYWORDS
S. Abortus equi, E. coli, bactericidal activity, complete genome, function of ORFs,
abortive rate, phage vB_SabS_Sds2 (Sds2)
Frontiers in Microbiology 01 frontiersin.org
fmicb-13-938616 August 10, 2022 Time: 14:20 # 2
Liu et al. 10.3389/fmicb.2022.938616
Introduction
Salmonella is a genus of the family Enterobacteriaceae that
is important for public health and economic development
worldwide. Diﬀerent serotypes of Salmonella often have
diﬀerent host speciﬁcity (Uzzau et al., 2000). Salmonella enterica
subspecies enterica serovar abortus equi (S. Abortus equi) is a
serotype restricted to equines, which is regarded as the most
common cause of abortion in mare

## test nodes

In [19]:
len(library.nodes)

32

In [20]:
pprint.pprint(library.nodes[30].metadata)

{'citation': 'Laszlo, Andrew H., et al. "Nanopore sequencing of the phi X 174 '
             'genome." Nanopore sequencing of the phi X 174 genome, 2024, pp. '
             '1-4.',
 'excerpt_keywords': 'Alignment, scaffolded assembly, short DNA sequences, '
                     'Illumina read, nanopore read, alignment quality, cutoff '
                     'filter, viral alignment, viral identification, viral '
                     'database, alignment speed, alignment confidence, DNA '
                     'identification, nanopore reads, alignment scores, SNP '
                     'calling, SNP detection, reference consensus, '
                     'transversions, transitions, confusion matrix, detection '
                     'efficiencies, resequencing.',
 'file_name': 'data/1406.4214.pdf',
 'section_summary': 'The key topics of this section are nanopore sequencing of '
                    'the phi X 174 genome, alignment of short DNA sequences, '
                    'viral alignm

In [10]:
library.documents[2]

Document(id_='c2089090-ef71-4ec9-9a22-e55fbfc32a6d', embedding=None, metadata={'page_label': '2', 'file_name': '1406.4214.pdf', 'file_path': 'data/1406.4214.pdf', 'file_type': 'application/pdf', 'file_size': 2841196, 'creation_date': '2024-01-17', 'last_modified_date': '2024-01-17', 'last_accessed_date': '2024-01-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='60fc5c86f922437eef7da25d912c7c9bcb5b797fb445ee542749f13580a17811', text=' 2 nucleotide resolution and control of the DNA translocation. Single -nucleotide resolution was \nrecently enabled through the development of MspA , a protein pore with a short and narrow \nconstriction (11, 13, 15, 16 ). DNA translocation control was also recently enabled through the \nuse of molecular motors such as p


## Extractor for abstract for each document

In [30]:
from llama_index.llms import OpenAI

llm = OpenAI(model='gpt-4')

In [35]:
from llama_index import PromptTemplate

extract_abstract_prompt_tpl = """
This is the first page of a scientifc paper. 
The task is to identify the paragraph corresponding to the abstract of the document. If no abstract found, try and identify the overview of the document.
The output required is :
    "abstract" : <abstract (identical to original text) or an overview of the document>
    "keywords": <up to 10 keywords identified in the abstract>
Example:
=== Page ===
1 Title: Nanopore sequencing of the phi X 174 genome

Authors: Andrew H. Laszlo1, Ian M. Derrington1, Brian C. Ross1, Henry Brinkerhoff1, Andrew
Adey2, Ian C. Nova1, Jonathan M. Craig1, Kyle W. Langford1, Jenny Mae Samson1, Riza Daza2, Kenji Doering1, Jay Shendure2, Jens H. Gundlach1,†

Affiliations: 1Department of Physics, University of Washington, Seattle, WA 98195, USA
2Department of Genome Sci ences, University of Washington, Seattle, WA 98195, USA
†Corresponding author. E -mail: gundlach@uw.edu

Abstract : Nanopore sequencing of DNA is a single -molecule technique that may achieve long reads, low cost, and high speed with minimal sample preparation and instrumentation. Here, we build on recent progress with respect to nanopore resolution and DNA control to interpret the procession of ion current levels observed during the translocation of DNA through the pore MspA. As approximately four nucleotides affect the ion current of each level, we measured the ion current corresponding to all 256 four -nucleotide comb inations (quadromers). This quadromer map is highly predictive of ion current levels of previously unmeasured sequences derived from the bacteriophage phi X 174 genome. Furthermore, we show nanopore sequencing reads of phi X 174 up to 4,500 bases in length that can be unambiguously aligned to the phi X 174 reference genome, and demonstrate proof -of-concept utility with respect to hybrid genome assembly and polymorphism detection. All methods and data are made fully available.

Main Text : DNA sequencing is revolutionizing biomedical and other life sciences research through its expanding scope (1) and has a rapidly growing presence in clinical medicine (2). These developments are driven in part by the successful completion of the Human Genome Project (3) and in part by the introduction of new sequencing technologies that have dramatically reduc ed the cost of DNA sequencing (4). Although such ‘next -generation’ sequencing
technologies have matured considerably since early proof -of-concept s (5-7), nearly all remain
limited to short sequence reads (with the exception of real -time sequencing from elongating polymerases (8)) and rely on complex, expensive instrumentation. Most platforms are also limited with respect to speed and require extensive sample preparation steps prior to sequencing .

Nanopore sequencing, independently proposed by Church and Deamer in the mid -1990s, has tremendous potential to overcome these limitations and achieve long reads, low cost, and high speed while requiring minimal sample preparation and instrumentation (i.e. ‘tricorder’ -like DNA sequencing devices) (9-12). However, this promise has faced substantial technical challenges, such that despite nearl y 20 years of effort, nanopore -derived sequence reads that align to complex, natural DNA sequences have yet to be demonstrated.

In nanopore devices directed at DNA sequencing, a salt solution is divide
=== Abstract ===
"abstract" : "Nanopore sequencing of DNA is a single -molecule technique that may achieve long reads, low cost, and high speed with minimal sample preparation and instrumentation. Here, we build on recent progress with respect to nanopore resolution and DNA control to interpret the procession of ion current levels observed during the translocation of DNA through the pore MspA. As approximately four nucleotides affect the ion current of each level, we measured the ion current corresponding to all 256 four -nucleotide comb inations (quadromers). This quadromer map is highly predictive of ion current levels of previously unmeasured sequences derived from the bacteriophage phi X 174 genome. Furthermore, we show nanopore sequencing reads of phi X 174 up to 4,500 bases in length that can be unambiguously aligned to the phi X 174 reference genome, and demonstrate proof -of-concept utility with respect to hybrid genome assembly and polymorphism detection. All methods and data are made fully available.",
"keywords" : "Nanopore sequencing", "DNA", "translocation", "low cost", "quadromer map", "genome", "polymorphism"

Your turn
=== Page ===
{page}

"""
extract_abstract_prompt = PromptTemplate(extract_abstract_prompt_tpl)


In [36]:
extract_abstract_prompt.dict()

{'metadata': {'prompt_type': <PromptType.CUSTOM: 'custom'>},
 'template_vars': ['page'],
 'kwargs': {},
 'output_parser': None,
 'template_var_mappings': None,
 'function_mappings': None,
 'template': '\nThis is the first page of a scientifc paper. \nThe task is to identify the paragraph corresponding to the abstract of the document. If no abstract found, try and identify the overview of the document.\nThe output required is :\n    "abstract" : <abstract (identical to original text) or an overview of the document>\n    "keywords": <up to 10 keywords identified in the abstract>\nExample:\n=== Page ===\n1 Title: Nanopore sequencing of the phi X 174 genome\n\nAuthors: Andrew H. Laszlo1, Ian M. Derrington1, Brian C. Ross1, Henry Brinkerhoff1, Andrew\nAdey2, Ian C. Nova1, Jonathan M. Craig1, Kyle W. Langford1, Jenny Mae Samson1, Riza Daza2, Kenji Doering1, Jay Shendure2, Jens H. Gundlach1,†\n\nAffiliations: 1Department of Physics, University of Washington, Seattle, WA 98195, USA\n2Departmen

In [37]:
abstract_gen = llm.predict(extract_abstract_prompt, page = library.documents[3].text[:4000])

In [39]:
display(Markdown(abstract_gen))

"abstract" : "Salmonella enterica subspecies enterica serovar abortus equi (S. Abortus equi) is the most common cause of abortion in mares. It has recently been found to cause abortion in donkeys more frequently in China. A novel virulent bacteriophage vB_SabS_Sds2 (hereafter designated as Sds2) was isolated from the feces of donkeys using a S. Abortus equi strain as a host. Phage Sds2 had an isometric polyhedral head and an uncontracted long tail, belonging to the Tequintavirus, Markadamsvirinae, Demerecviridae, Caudovirales. The genome of phage Sds2 was 114,770 bp, with a GC content of 40.26%. The genome contained 160 open reading frames (ORFs), and no ORFs were associated with pathogenicity, drug resistance, or lysogenization by sequence analysis. Both genome annotation and phylogenetic analysis indicated that phage Sds2 was highly similar to T5-like bacteriophages. Phage Sds2 could lyse 100% (30/30) of S. Abortus equi strains, 25.3% (24/95) of other serotypes ofSalmonella strains, and 27.6% (8/29) of Escherichia coli strains using the double-layer agar plate method. The in vitro test showed that phage Sds2 had high bactericidal activity against S. Abortus equi at a wide range of MOIs. The in vivo test indicated that phage Sds2 had an inhibitory effect on abortion in mice challenged with S. Abortus equi. In general, phage Sds2 is a novel lytic phage with a wide host range and has the potential to prevent abortion caused byS. Abortus equi.",
"keywords" : "Salmonella enterica", "abortion", "donkeys", "bacteriophage vB_SabS_Sds2", "Tequintavirus", "genome", "open reading frames", "T5-like bacteriophages", "Escherichia coli", "bactericidal activity"

## Test citation prompt

In [ ]:
from phages.modules.prompts import citation_prompt
from datetime import datetime

current_year = datetime.now().year

cit = citation_prompt.partial_format(year=current_year)

print(citation_prompt.json())

response = llm.predict(cit, text = text)

response

{"metadata": {"prompt_type": "custom"}, "template_vars": ["year", "text"], "kwargs": {}, "output_parser": null, "template_var_mappings": null, "function_mappings": null, "template": "Provide the citation for the following text in MLA Format. The year is {year}\n\n    {text}\n\n\n    Citation:"}
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': <MessageRole.USER: 'user'>, 'content': 'Provide the citation for the following text in MLA Format. The year is 2024\n\n    \nProc. Natl. Acad. Sci. USA\nVol. 80, pp. 4195-4199, July 1983\nBiochemistry\nIn vitro synthesis of bacteriophage 4X174 by purified components\n(in vitro DNA synthesis/viral component/DNA polymerase Ill holoenzyme/rep protein/deoxyuridinetriphosphatase)\nAKIRA AOYAMAt, ROBERT K. HAMATAKEt, AND MASAKI HAYASHIt\nDepartments of tBiology and tChemistry, University of California at San Diego, La Jolla, California 92093\nCommunicated by E. P

'Aoyama, Akira, et al. "In vitro synthesis of bacteriophage 4X174 by purified components." Proceedings of the National Academy of Sciences of the United States of America, vol. 80, no. 14, 1983, pp. 4195-4199.'

In [ ]:
citation_prompt.template_vars

['year', 'text']

## Test FullPDFReader

In [ ]:
from phages.modules.library import FullPDFReader

# for SimpleDirectoryReader file_extractor param
CUSTOM_FILE_READER_CLS = {
    ".pdf": FullPDFReader,
}

file_suffix=".pdf"

file_extractor=CUSTOM_FILE_READER_CLS

reader = file_extractor[file_suffix]

In [8]:


response = reader.load_data(file='data/1406.4214.pdf')

len(response)

1

## Test adoc match

In [11]:
from phages.modules.library import Library
from pathlib import Path

library = Library()

# Define the directory path
directory_path = Path('/workspaces/ml-learning/src/phages/data/test3')

# Loop over each file in the directory
for file_path in directory_path.iterdir():
    # Check if it's a file and not a directory
    if file_path.is_file():
        # Call the library.add() function with the file path as a string
        library.add(str(file_path))


embedded weaviate is already listening on port 8079
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{"action":"restapi_management","level":"info","msg":"Shutting down... ","time":"2024-01-23T09:25:38Z"}
{"action":"restapi_management","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-01-23T09:25:38Z"}


Multiple definitions in dictionary at byte 0x32487 for key /MediaBox
Multiple definitions in dictionary at byte 0x327a8 for key /MediaBox
Multiple definitions in dictionary at byte 0x32a79 for key /MediaBox
Multiple definitions in dictionary at byte 0x32c82 for key /MediaBox
Multiple definitions in dictionary at byte 0x32e3c for key /MediaBox
Multiple definitions in dictionary at byte 0x32fee for key /MediaBox
Multiple definitions in dictionary at byte 0x331d8 for key /MediaBox
Multiple definitions in dictionary at byte 0x333ca for key /MediaBox
Multiple definitions in dictionary at byte 0x3365b for key /MediaBox
Multiple definitions in dictionary at byte 0x338ac for key /MediaBox
Multiple definitions in dictionary at byte 0x33a86 for key /MediaBox
Multiple definitions in dictionary at byte 0x33c88 for key /MediaBox
Multiple definitions in dictionary at byte 0x33e79 for key /MediaBox
Multiple definitions in dictionary at byte 0x34102 for key /MediaBox
Multiple definitions in dictionary

{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-23T09:25:47Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-23T09:25:47Z"}
{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-23T09:25:47Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-23T09:25:47Z"}


listen tcp :6060: bind: address already in use


{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"docs_SEbBamV3puOr","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-23T09:25:47Z","took":174300}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-01-23T09:25:48Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-01-23T09:25:48Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2024-01-23T09:25:48Z"}
/usr/local/lib/python3.11/subprocess.py:1127: ResourceWarning: subprocess 17092 is still running
  _warn("subprocess %s is still running" % self.pid,
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"nodes_yshPFQSIWFOC","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-23T09:25:48Z","took":154211400}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Multiple definitions in dictionary at byte 0x21878 for key /MediaBox
Multiple definitions in dictionary at byte 0x21b0c for key /MediaBox
Multiple definitions in dictionary at byte 0x21e55 for key /MediaBox
Multiple definitions in dictionary at byte 0x22023 for key /MediaBox
Multiple definitions in dictionary at byte 0x221d9 for key /MediaBox
Multiple definitions in dictionary at byte 0x223aa for key /MediaBox
Multiple definitions in dictionary at byte 0x22538 for key /MediaBox
Multiple definitions in dictionary at byte 0x226f1 for key /MediaBox
Multiple definitions in dictionary at byte 0x22957 for key /MediaBox
Multiple definitions in dictionary at byte 0x22b98 for key /MediaBox
Multiple definitions in dictionary at byte 0x22e09 for key /MediaBox
New documents count: 1
calling get_citation
citation prompt: metadata={'prompt_typ

In [12]:
from phages.modules.helpers import pickleSave

pickleSave(library, 'library_with_original_phages', '.')

Saving object library_with_original_phages to pickle file ./library_with_original_phages.pkl


In [13]:
len(library.documents)

6

In [15]:
library.documents[0].metadata

{'file_path': '/workspaces/ml-learning/src/phages/notebooks/data/dataset_phages-microbiology-frontiers_2024-01-23_08-51-12-772.json',
 'file_name': 'dataset_phages-microbiology-frontiers_2024-01-23_08-51-12-772.json',
 'file_type': 'application/json',
 'file_size': 993570,
 'creation_date': '2024-01-23',
 'last_modified_date': '2024-01-23',
 'last_accessed_date': '2024-01-23',
 'citation': '"Two New Lytic Bacteriophages of the Myoviridae Family Against Carbapenem-Resistant Acinetobacter baumannii." Frontiers in Microbiology, vol. 9, 2018, doi:10.3389/fmicb.2018.00850.'}

In [17]:
brol = await library.adoc_match('agricultural disease')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [20]:
brol[0].get_content()

'Liao, Yen-Te, et al. "Discovery of Shiga Toxin-Producing Escherichia coli (STEC)-Specific Bacteriophages From Non-fecal Composts Using Genomic Characterization." Frontiers in Microbiology, vol. 10, 2019, p. 627. doi: 10.3389/fmicb.2019.00627.'

In [21]:
brol[1].get_content()

'Tan, C. W., Rukayadi, Y., Hasan, H., Abdul-Mutalib, N.-A., Jambari, N. N., Hara, H., Thung, T. Y., Lee, E., & Radu, S. (2021). Isolation and Characterization of Six Vibrio parahaemolyticus Lytic Bacteriophages From Seafood Samples. Frontiers in Microbiology, 12, 616548. doi: 10.3389/fmicb.2021.616548'

{"action":"read_disk_use","level":"error","msg":"failed to read disk usage: no such file or directory","path":"./weaviate/.local/share/weaviate","time":"2024-01-24T09:14:35Z"}
{"action":"read_disk_use","level":"error","msg":"failed to read disk usage: no such file or directory","path":"./weaviate/.local/share/weaviate","time":"2024-01-24T09:14:36Z"}
{"action":"read_disk_use","level":"error","msg":"failed to read disk usage: no such file or directory","path":"./weaviate/.local/share/weaviate","time":"2024-01-24T09:14:36Z"}


## Test json loader

In [7]:
from pathlib import Path
from llama_index import download_loader

JSONReader = download_loader("JSONReader")

loader = JSONReader()
documents = loader.load_data()

/workspaces/ml-learning/.venv/lib/python3.11/site-packages/pkg_resources/_vendor/jaraco/text/__init__.py:593: ResourceWarning: unclosed file <_io.TextIOWrapper name='/workspaces/ml-learning/.venv/lib/python3.11/site-packages/llama_index/download/llamahub_modules/requirements.txt' mode='r' encoding='UTF-8'>
  for item in lines:


In [2]:
documents[0].metadata

{}

In [3]:
from phages.modules.library import Library
library = Library()
library.add('/workspaces/ml-learning/src/phages/notebooks/data/dataset_phages-microbiology-frontiers_2024-01-23_08-51-12-772.json')

embedded weaviate is already listening on port 8079
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
New documents count: 1
calling get_citation
citation prompt: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['year', 'text'] kwargs={'year': 2024} output_parser=None template_var_mappings=None function_mappings=None template='Provide the citation for the following text in MLA Format. The year is {year}\n\n    {text}\n\n\n    Citation:'
text: [{
  "url": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",
  "crawl": {
    "loadedUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",
    "loadedTime": "2024-01-23T

True

In [6]:
library.documents[0].text

'[{\n  "url": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",\n  "crawl": {\n    "loadedUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",\n    "loadedTime": "2024-01-23T08:48:06.942Z",\n    "referrerUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",\n    "depth": 0,\n    "httpStatusCode": 200\n  },\n  "metadata": {\n    "canonicalUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",\n    "title": "Articles | Frontiers",\n    "description": null,\n    "author": null,\n    "keywords": null,\n    "languageCode": "en"\n  },\n  "screenshotUrl": null,\n  "text": "Original Research\\nPublished on 30 Apr 2018\\nTwo New Lytic Bacteriophages of the Myoviridae Family Against Carbapenem-Resistant Acinetobacter baumannii\\nFrontiers in Microbiology\\ndoi 10.3389/fmicb.2018.00850\\n7,708 views \\n45 citati

### First load data to a dictionary

In [2]:
import json
    # Replace 'your_file.txt' with the path to your text file
file_path = '/workspaces/ml-learning/src/phages/notebooks/data/dataset_phages-microbiology-frontiers_2024-01-23_08-51-12-772.json'



def jsonToDict(file_path: str)-> dict:


    # Read the JSON object from the file
    with open(file_path, 'r') as file:
        data = file.read()

    # Parse the JSON content
    try:
        json_data = json.loads(data)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        json_data = None

    # Output the parsed data
    if json_data is not None:
        print(json.dumps(json_data, indent=4))

    return json_data


In [3]:
json_data = jsonToDict(file_path)

[
    {
        "url": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",
        "crawl": {
            "loadedUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",
            "loadedTime": "2024-01-23T08:48:06.942Z",
            "referrerUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",
            "depth": 0,
            "httpStatusCode": 200
        },
        "metadata": {
            "canonicalUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",
            "title": "Articles | Frontiers",
            "description": null,
            "author": null,
            "keywords": null,
            "languageCode": "en"
        },
        "screenshotUrl": null,
        "text": "Original Research\nPublished on 30 Apr 2018\nTwo New Lytic Bacteriophages of the Myoviridae Family Against Carbapenem-Resis

### Then use json loader "JSONDataReader" from llama_hub

In [6]:
from pathlib import Path
from llama_index import download_loader

JSONReader = download_loader("JSONReader")

loader = JSONReader(levels_back=0)


In [7]:
json_file_path = '/workspaces/ml-learning/src/phages/notebooks/data/dataset_phages-microbiology-frontiers_2024-01-23_08-51-12-772.json'

In [8]:
documents = loader.load_data(json_file_path)

In [9]:
documents[0].text

'{\n"url": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",\n"crawl": {\n"loadedUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",\n"loadedTime": "2024-01-23T08:48:06.942Z",\n"referrerUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research",\n"depth": 0,\n"httpStatusCode": 200\n"metadata": {\n"canonicalUrl": "https://www.frontiersin.org/articles?domain=all&query=bacteriophages&type=original-research#",\n"title": "Articles | Frontiers",\n"description": null,\n"author": null,\n"keywords": null,\n"languageCode": "en"\n"screenshotUrl": null,\n"text": "Original Research\\nPublished on 30 Apr 2018\\nTwo New Lytic Bacteriophages of the Myoviridae Family Against Carbapenem-Resistant Acinetobacter baumannii\\nFrontiers in Microbiology\\ndoi 10.3389/fmicb.2018.00850\\n7,708 views \\n45 citations \\nOriginal Research\\nPublished on 4 Jan 2018\\nMetavirome Seq

 Parsing xml https://blog.apify.com/python-parse-xml/

 Pdf download https://www.frontiersin.org/articles/10.3389/fmicb.2018.00850/pdf?isPublishedV2=False



In [ ]:
"""Custom JSON Reader."""

import json
import re
from pathlib import Path
from typing import Dict, Generator, List, Optional

from llama_index.readers.base import BaseReader
from llama_index.readers.schema.base import Document

class CustomJSONReader(BaseReader):